# Explain a prediction

This model demonstrates the use of the CHAMOIS API to establish links between the genes of a query cluster and the ChemOnt classes of the putative metabolite as predicted by CHAMOIS.

In [ ]:
import chamois
chamois.__version__

## Loading data

Use `gb-io` to load the GenBank record for a cluster into a dedicated `ClusterSequence` object. Let's use [AB746937.1](https://www.ncbi.nlm.nih.gov/nuccore/AB746937.1), the biosynthetic gene cluster for [muraminomicin](https://pubchem.ncbi.nlm.nih.gov/compound/145720725) found in *Streptosporangium amethystogenes*. 

In [ ]:
import gb_io
import chamois.model
records = gb_io.load("data/AB746937.1.gbk")
clusters = [chamois.model.ClusterSequence(records[0])]

## Calling genes

You can use the `chamois.orf` module to call the genes inside one or more `ClusterSequence` objects. Since the source GenBank record has already gene called (in `CDS` features, with the gene name added in the `/protein_id` qualifier), we can skip gene calling and simply extract the already-present genes. For this, we use a `CDSFinder`: 

In [ ]:
from chamois.orf import CDSFinder
orf_finder = CDSFinder(locus_tag="protein_id")
proteins = list(orf_finder.find_genes(clusters))

## Extracting features

Once we have a list of proteins, we need to annotate them with protein domains. CHAMOIS is distributed with the Pfam HMMs required by the CHAMOIS predictor, so we can simply use these and run the default annotation with a `PfamAnnotator` object: 

In [ ]:
from chamois.domains import PfamAnnotator
annotator = PfamAnnotator()
domains = list(annotator.annotate_domains(proteins))

## Building compositional matrices

We now have a list of domains, but we want to turn these domains into a matrix of presence/absence of each Pfam domain in each gene cluster. To do so, let's first load the trained CHAMOIS predictor, so we know which features we need to extract: 

In [ ]:
from chamois.predictor import ChemicalOntologyPredictor
predictor = ChemicalOntologyPredictor.trained()

Then simply build the observations table (from the source clusters), and the actual compositional data matrix, returned as an `AnnData` object to preserve observation and feature metadata:

In [ ]:
import chamois.compositions 
obs = chamois.compositions.build_observations(clusters)
data = chamois.compositions.build_compositions(domains, obs, predictor.features_)
data

In [ ]:
data.var_vector(clusters[0].id)

## Infer chemical classes

With the compositional matrix ready, we can simply call the `predict_probas` method on the predictor to get the class probabilities predicted by CHAMOIS:

In [ ]:
probas = predictor.predict_probas(data)

`probas` is a NumPy array containing probabilities for each of the classes of the model. We can turn these predictions into a table retaining the metadata from the original predictor:

In [ ]:
classes = predictor.classes_.copy()
classes['probability'] = probas[0]
classes[classes['probability'] > 0.5]

## Build gene contribution table

Now that we have the predictions, we can inspect the model to explain which genes of the cluster contributed to the prediction of each class. This can be done in the command line with the `chamois explain cluster` subcommand, or programmatically:

In [ ]:
from chamois.cli.explain import build_genetable
genetable = build_genetable(proteins, domains, predictor, probas).set_index("class")
genetable

## Render cluster

Now that we have a table summarizing the role of every cluster gene in the prediction of each ChemOnt class, we can render the genomic locus of the BGC with additional information about the function of each gene. Let's restrict to 5 specific classes with the lowest amount of training examples in MIBiG 3.1:  

In [ ]:
top = predictor.classes_.loc[genetable.index].sort_values("n_positives").head(5).index
predictor.classes_.loc[top]

We can now plot the cluster while colouring the genes according to which ChemOnt class they contribute the most, highlighting their function in the biosynthetic pathway. For the display, let's use the `dna-features-viewer` library.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from dna_features_viewer import GraphicFeature, GraphicRecord
from palettable.cartocolors.qualitative import Vivid_10
from palettable.cartocolors.sequential import *

# build a palette
palette = dict(zip(top, [Purp_2, Sunset_5, DarkMint_2, Magenta_2, Teal_5, BluGrn_2]))
fig = plt.figure(figsize=(12, 6))

# extract CDS features from the record
features = []
for feature in filter(lambda f: f.kind == "CDS", records[0].features):
    # get the name and product of the gene
    label = next(q.value for q in feature.qualifiers if q.key == "protein_id")
    product = next((q.value for q in feature.qualifiers if q.key == "product"), None)
    if product.startswith("putative"):
        product = product[9:]
    # get the coordinates
    start = feature.location.start
    end = feature.location.end
    if feature.location.strand == "-":
        start, end = end, start
    # get the colour of the gene based on contribution weight
    weights = genetable[label].loc[top]
    if any(weights >= 1):
        best = weights.index[weights.argmax()]
        color = palette[best].hex_colors[1]
    else:
        color = "#c0c0c0"
    # record the feature
    features.append(GraphicFeature(
        start=start,
        end=end,
        strand=-1 if feature.location.strand == "-" else 1,
        color=color,
        label=None if color == "#c0c0c0" else product,
    ))

# render the feature records
record = GraphicRecord(sequence=records[0].sequence, features=features)
record.plot(ax=plt.gca())

# add legend
legend_elements = [
    Patch(
        facecolor=v.hex_colors[1], 
        edgecolor='black', 
        label=f"{k} - {predictor.classes_.name.loc[k]}"
    )
    for k,v in palette.items()
]

# create the figure
plt.legend(handles=legend_elements, loc='upper left')
plt.title("AB746937.1 - muraminomicin")
fig.tight_layout()